# The Unsent Project Data Collection

## Import packages

In [40]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
from collections import defaultdict

## Web scraping

In [2]:
data = defaultdict(lambda: defaultdict())

In [1]:
#link = 'https://theunsentproject.com/2020/06/emily-%e6%9d%8e%e6%95%8f/' # arbitrary first one
#link = 'https://theunsentproject.com/2020/06/hannah-p-3/' # the one before elmily
#link = 'https://theunsentproject.com/2020/06/m-3212/'
#link = 'https://theunsentproject.com/2020/06/roderick-2/'
#link = 'https://theunsentproject.com/2019/09/i-v-2/'
link = 'https://theunsentproject.com/2019/06/thevampirelover/'
#link = 'https://theunsentproject.com/2020/02/andrew-391/' # start tom then go back to other

In [4]:
i = 0

### Starting at given link, traverse posts and scrape data

In [23]:
#writer = csv.writer(open('D:/Coding/Projects/Python/The Unsent Project/unsent_data.csv', 'a', newline=''))
while True:
    # http request to page
    page = requests.get(link)
    inside_html = BeautifulSoup(page.content, 'html.parser')

    # extracting elements
    ps = inside_html.findAll('p')
    headers = inside_html.findAll('aside', class_='post-meta cf')
    titles = inside_html.findAll('h1')
    
    prev_links = inside_html.findAll('a', class_='post_nav_link prev')
    next_links = inside_html.findAll('a', class_='post_nav_link next')
    
    # indicates error connecting
    if (not headers):
        print (i)
        continue
    
    # extract the data from those elements
    name = titles[0].get_text()
    date_color = headers[0].get_text()
    date, color = date_color.split('/')
    message = ps[3].get_text()
    
    if (prev_links):
        prev_link = prev_links[0].get('href')
    else:
        print (i)
        break
    if (next_links):
        next_link = next_links[0].get('href')
    else:
        print (i)
        break
    
    # clean the data
    date = date.replace('\n\t\t', '')
    date = date.rstrip(' ')
    color = color.strip(' ')
    
    # store in dictionary
    data[i] = {
        'name':name,
        'date':date,
        'color':color,
        'message':message
    }
    
    #row = [str(value.encode('utf-8')).strip('b').strip('\'') for value in data[i].values()]
    #writer.writerow(row)
    
    i += 1
    link = prev_link
    #link = next_link

10099
10099
10099
10099
10099
10099
10099
10099
10130
10130
10130
10130
10209
10219
10225
10226
10226
10227
10227
10228
10228
10228
10228
10228
10228
10228
10228
10228
10228
10228


KeyboardInterrupt: 

#### Look through the data to chec if things are working and to debug

In [56]:
data[0]

{'name': 'H.J.M.',
 'date': 'December 20, 2019',
 'color': 'Tangerine',
 'message': 'Every night, I sleep like a baby, thanks to you.\n??❤️?'}

In [18]:
i

10100

In [19]:
len(data)

10100

In [20]:
data[len(data) - 1]
#data[10536]

{'name': 'V.H.',
 'date': 'May 15, 2020',
 'color': 'Blue',
 'message': 'You’re so special and unique. I wish I could show you that every day. I don’t want you to feel like you’re not good enough. You take my breath away.'}

In [18]:
# Given a name, loop through all instances until we get to the one closest to the post we want to check
for j in range(654, 697):
    link = f'https://theunsentproject.com/2020/06/chris-{j}/'
    page = requests.get(link)
    inside_html = BeautifulSoup(page.content, 'html.parser')

    # extracting elements
    ps = inside_html.findAll('p')
    
    # extract the data from those elements
    message = ps[3].get_text()
    
    if ('bad bunny' in message):
        print (j)
        break

682


In [22]:
link

'https://theunsentproject.com/2020/05/r-k/'

### Write to a file

In [21]:
writer = csv.writer(open('D:/Coding/Projects/Python/The Unsent Project/unsent_data8.csv', 'w', newline=''))
columns = ['name', 'date', 'color', 'message']
writer.writerow(columns)
for i in range(len(data)):
    # utf-8 encoding in hindsight was a lazy solution that costed me later
    row = [str(value.encode('utf-8')) for value in data[i].values()]
    writer.writerow(row)

## Cleaning

In [143]:
# get all that data
header = ['name', 'date', 'color', 'message']
data_dict = defaultdict(lambda: [])
for col in header:
    data_dict[col] = []
df = pd.DataFrame(data=data_dict)
for i in range(1, 7):
    df_temp = pd.read_csv('D:/Coding/Projects/Python/The Unsent Project/unsent_data' + str(i) + '.csv')
    df = pd.concat([df, df_temp])
df_temp = pd.read_csv('D:/Coding/Projects/Python/The Unsent Project/unsent_data7justjuly.csv')
df = pd.concat([df, df_temp])

In [117]:
df.head()['message'][0]

'i miss u. i loved u. i never knew that well""\\\\r\\\\n"\\r\\n'

In [ ]:
for i, row in list(df.iterrows())[:5]:
    print (row['message'])

In [144]:
import re
end1 = '""\\r\\n"\r\n'
end2 = '\\\\n\\r\\n\r\n' # basically I fucked up when writing to the csv
def unfuck(s):
    s = str(s).rstrip(end1).rstrip(end2)
    new_s = ''
    n = 0
    for i, c in enumerate(reversed(s)):
        if c == '\\':
            if (new_s[n - 2:n] != 'x\\' and new_s[n - 2:n] != 'n\\'):
                new_s += c
                n += 1
        else:
            new_s += c
            n += 1
    new_s = new_s[::-1]
    new_s = new_s.replace('\\xe2\\x80\\x99', '\'')
    new_s = new_s.replace('\\xf0\\x9f\\x92\\x9e', '?')
    new_s = new_s.replace('\\xf0\\x9f\\x92\\x8b', '?')
    new_s = new_s.replace('\\xe2\\x80\\x94', '—')
    new_s = new_s.replace('\\xe2\\x80\\xa6', '...')
    new_s = new_s.replace('\\xe2\\x80\\x9c', '"')
    new_s = new_s.replace('\\xe2\\x80\\x9d', '"')
    new_s = new_s.replace('\\xe2\\x80\\x93', '-')
    new_s = new_s.replace('\\n', ' ')
    new_s = re.sub(r'\\*x[a-z| |0-9][a-z| |0-9]', '', new_s)
    '''
    new_s = str(s)
    new_s = re.sub(r'\\*x[a-z| |0-9][a-z| |0-9]', '', new_s)
    new_s = new_s.replace('""\\r\\n"', '')
    new_s = re.sub(r'"+\\+[nr]', '', new_s)
    new_s = new_s.rstrip()
    new_s = ''.join([x for x in new_s if ord(x) < 127])
    '''
    return new_s

def name_unfuck(s):
    return re.sub(r'\\*x[a-z| |0-9][a-z| |0-9]', '', str(s))

df['message'] = df['message'].apply(lambda x: unfuck(x))
df['name'] = df['name'].apply(lambda x: name_unfuck(x))

In [131]:
len(df)

228901

In [53]:
colors = ['Army Green','Black','Blue','Blue Grey','Brown','Dark Blue','Dark Green','Dark Purple','Green','Grey',\
          'Light Blue','Light Green','Light Grey','Light Orange','Light Pink','Light Purple','Maroon','Orange',\
          'Pale Blue','Pale Pink','Pale Purple','Peach','Pink','Purple','Red','Tan','Tangerine','Turquoise','White',\
          'Wine','Yellow']

In [145]:
df = df[df['color'].isin(colors)]
df = df[df['date'] != 'July 1 2020']

In [146]:
len(df)

199931

In [147]:
df.head()

,name,date,color,message
1237,Evan,June 30 2020,Tangerine,All I want is you to be happy. I love you fore...
1238,Procureur,June 30 2020,Light Pink,Tu es mon meilleur ami
1239,shiv,June 30 2020,Pale Purple,i loved you... more than you ever cared for me...
1240,brendan,June 30 2020,Yellow,you make me so incredibly happy & im so in lov...
1241,James,June 30 2020,Yellow,There this thing about you. No matter how many...


In [136]:
set(df['message'])

{'',
 'i can feel you leaving again, i wish you would just say something.',
 'i really dont know why i broke up with you and sometimes i look at your contact ready to teyou but i cant.',
 "It's been 7 years and after all this time",
 'it was never romantic but i love you so much and it hurts so much more - E',
 'i hate that i still miss you',
 "i didn't know how much i needed you until i couldn't have you.",
 "we'll be together again. one day. i know we will. please don't let go. -G",
 "I really do hope that distance won't be the reason why I'll lose you.",
 "your name doesn't start with A. ",
 'you broke my heart. you never said you were sorry and you knew how much you had hurt me.',
 "i was lying when i said i wasn't manifesting. also, i'm baby",
 "Im sorry I'm not sober again, but it's not like you'd care",
 'you were the first guy I ever actually loved, not your looks but everything else. I miss you more and more every time I talk to another guy. I wish. there was a way to go back 

In [148]:
df.to_csv('D:/Coding/Projects/Python/The Unsent Project/unsent_data_final2.csv', index=False, header=True)